In [ ]:
import json
from __future__ import annotations
from abc import ABC, abstractmethod
import random
import pandas as pd

In [ ]:

ROCK = "rock"
SCISSORS = "scissors"
PAPER = "paper"
LIZARD = "lizard"
SPOCK = "spock"

CHOICES = [ROCK, SCISSORS, PAPER, LIZARD, SPOCK]


RULES = {
    ROCK: {
        "win":  [SCISSORS, LIZARD],
        "lose": [PAPER, SPOCK],
        "symbol":"RK"
    },

    PAPER: {
        "win":  [ROCK, SPOCK],
        "lose": [SCISSORS, LIZARD],
        "symbol":"PA"
    },

    LIZARD: {
        "win": [PAPER, SPOCK],
        "lose": [SCISSORS, ROCK],
        "symbol":"LZ"
    },
    SPOCK:{
        "win":  [ROCK, SCISSORS],
        "lose": [PAPER, LIZARD],
        "symbol":"SP"
    },
    SCISSORS: {
        "win":[PAPER, LIZARD],
        "lose": [ROCK, SPOCK],
        "symbol":"SC"
    }
}



In [ ]:



def find(pred, iterable):
    for element in iterable:
        if pred(element):
            return element
    return None


def readFile(filename="src/data.json"):
    with open(filename, 'r') as f:
        data = json.load(f)
        return data


def writeFile(data, filename="src/data.json"):
    with open(filename, 'w') as f:
        json.dump(data, f)



In [ ]:


class IStrategy(ABC):
    @abstractmethod
    def do_algorithm(self, data,last_game, constant=None) -> str:
        pass


In [ ]:


#TODO Son 11 tipos de jugadores , y son 10 rounds, cada uno de los 11 jugadores juega contra los otros 10 y esto lo repites 100 veces

# * Jugar siempre lo mismo
class ConstantStrategy(IStrategy):
    def do_algorithm(self, data, last_game, constant) -> str:
        self.choice = constant
        return self.choice

# * Jugar al azar
class RandomStrategy(IStrategy):
    def do_algorithm(self, data, last_game, constant) -> str:
        self.choice = random.choice(data)
        return self.choice


# * Jugar siempre lo que tiró el contrario en el juego anterior
class OppositeStrategy(IStrategy):
    def do_algorithm(self, data, last_game, constant) -> str:
        self.choice = last_game["choice_opponent"]
        return self.choice


# * Jugar lo mismo si ganó, jugar lo que tiró el contrario si perdió
class SameIfWinStrategy(IStrategy):
    def do_algorithm(self, data, last_game, constant) -> str:
        if(last_game["status"] == 'victory'):
            self.choice = last_game['choice_player']
            return self.choice
        else:
            self.choice = last_game["choice_opponent"]
            return self.choice

# * Jugar lo mismo si perdió, jugar lo que tiró el contrario si ganó


class SameIfLoseStrategy(IStrategy):
    def do_algorithm(self, data: list, last_game=None, constant=None) -> str:
        if(last_game["status"] == 'loss'):
            self.choice = last_game['choice_player']
            return self.choice
        else:
            self.choice = last_game['choice_opponent']
            return self.choice

# * Jugar lo que derrotaría a la elección anterior del adversario


class WinOpponentPreviousChoiceStrategy(IStrategy):
    def do_algorithm(self, data: list, last_game=None, constant=None) -> str:
        arr = RULES[last_game["choice_opponent"]]["lose"]
        self.choice = random.choice(arr)
        return self.choice

# * Jugar lo sería derrotado dada la elección anterior del adversario


class LoseOpponentPreviousChoiceStrategy(IStrategy):
    def do_algorithm(self, data: list, last_game=None, constant=None) -> str:
        arr = RULES[last_game["choice_opponent"]]["win"]
        self.choice = random.choice(arr)
        return self.choice


# * Jugar suponiendo que el adversario sigue un ciclo (i.e. R, S, P, R, S, P,... )


class PatternOpponentStrategy(IStrategy):
    def gen(self,pattern_opponent):
        def init():
            return 0
        i = init()

        while True:
            val = (yield pattern_opponent[i])
            if val == "restart":
                i = init()
            else:
                i += 1

    def do_algorithm(self, data: list, last_game=None, constant=None) -> str:
        self.last = "";
        self.pattern_opponent = ["RK", "SC", "PA", "LZ", "SP"]
        g = self.gen(self.pattern_opponent)
        value = ""
        if(self.last == self.pattern_opponent[len(self.pattern_opponent)-1]):
            value = g.send("restart")
        else:
            value = g.__next__()
        x = find(lambda x: RULES[x]["symbol"] == value, RULES)
        choice = random.choice(RULES[x]["lose"])
        self.last = value
        return choice


STRATEGIES = {
    "constant": ConstantStrategy,
    "random":RandomStrategy,
    "opposite":OppositeStrategy,
    "sameIfWin":SameIfWinStrategy,
    "sameIfLose":SameIfLoseStrategy,
    "winOpponentPreviousChoice":WinOpponentPreviousChoiceStrategy,
    "loseOpponentPreviousChoice":LoseOpponentPreviousChoiceStrategy,
    "patternOpponent":PatternOpponentStrategy
}



In [ ]:

class Stats:
    def __init__(self):
        self.draws = 0
        self.wins = 0
        self.losses = 0
        self.total_points = 0
        self.games = []
        self.rating = []


In [ ]:

class Player:
    def __init__(self, strategy: IStrategy, initial_strategy, next_strategy, id, name, elo, constant) -> None:
        self.id = str(id)
        self.constant = constant
        self._strategy = strategy
        self.choices = CHOICES
        self.name = name
        self.elo = elo
        self.initial_strategy = initial_strategy
        self.next_strategy = next_strategy
        self.old_elo = elo
        self.stats = Stats()

    @property
    def strategy(self) -> IStrategy:
        return self._strategy

    @strategy.setter
    def strategy(self, strategy: IStrategy) -> None:
        self._strategy = strategy

    def pickOut(self) -> None:
        length = len(self.stats.games)
        if length > 0:
            self.updateStrategy()
        last_game = self.stats.games[length-1] if length > 0 else {}
        result = self._strategy.do_algorithm(
            self.choices, last_game, self.constant)
        self.getNameStrategy()
        return result

    def toJSON(self) -> dict:
        return{
            "id": self.id,
            "name": self.name,
            "elo": self.elo,
            "strategy": self.initial_strategy,
            "next_strategy": self.next_strategy,
            "constant": self.constant,
            "stats": {
                "wins": self.stats.wins,
                "losses": self.stats.losses,
                "draws": self.stats.draws,
                "total_points": self.stats.total_points,
                "games": self.stats.games,
                "rating": self.stats.rating
            }
        }

    def updateStrategy(self) -> None:
        self._strategy = STRATEGIES[self.next_strategy]()

    def getNameStrategy(self) -> str:
        name = self._strategy.__class__.__name__
        print(name)
        return name


In [ ]:

class Game:
    def __init__(self, number, player1: Player, player2: Player):
        self.number = number
        self.player1 = player1
        self.player2 = player2
        self.winner: Player = None
        self.loser: Player = None
        self.result = None

    def calculate_new_rating(self, player_rating, opponent_rating, result):
        K = 32
        expected_score_a = self.calculate_expected_score(
            player_rating, opponent_rating)
        new_rating_a = player_rating + K * (result - expected_score_a)
        return new_rating_a

    def calculate_expected_score(self, player_rating, opponent_rating):
        expected_score_a = 1 / \
            (1 + 10 ** ((opponent_rating - player_rating) / 400))
        return expected_score_a

    def getRating(self, player: Player, opponent: Player, result):
        elo = self.calculate_new_rating(player.elo, opponent.elo, result)
        expected_score = self.calculate_expected_score(
            player.elo, opponent.elo)
        return {
            "elo": round(elo),
            "expected_score": round(expected_score, 3),
        }

    def Draw(self):
        result = 0.5
        self.player1.elo = self.getRating(
            self.player1, self.player2, result)["elo"]
        self.player2.elo = self.getRating(
            self.player2, self.player1, result)["elo"]
        self.player1.stats.draws += 1
        self.player2.stats.draws += 1
        self.player1.stats.total_points += result
        self.player2.stats.total_points += result

    def calcStats(self):
        self.winner.elo = self.getRating(self.winner, self.loser, 1)["elo"]
        self.loser.elo = self.getRating(self.loser, self.winner, 0)["elo"]
        self.winner.stats.wins += 1
        self.loser.stats.losses += 1
        self.winner.stats.total_points += 1
        self.loser.stats.total_points += 0
        return

    def Start(self):
        self.choice_player1 = self.player1.pickOut()
        self.choice_player2 = self.player2.pickOut()
        print(self.choice_player1, "player 1: ", self.player1.name)
        print(self.choice_player2, "player 2: ", self.player2.name)
        if(self.choice_player1 == self.choice_player2):
            self.winner = None
            self.loser = None
            self.Draw()
        elif(self.choice_player1 in RULES[self.choice_player2]["win"]):
            self.winner = self.player2
            self.loser = self.player1
            self.calcStats()
        else:
            self.winner = self.player1
            self.loser = self.player2
            self.calcStats()

        print(self.player1.elo, "P1")
        print(self.player2.elo, "P2")
        if(self.winner):
            print("Winner: ", self.winner.name)
        else:
            print("Draw")

        return

    def getWinner(self):
        if (self.winner):
            return self.winner.name
        else:
            return "Draw"

    def getLoser(self):
        if (self.loser):
            return self.loser.name
        else:
            return "Draw"

    def Play(self):
        self.Start()

        data = readFile("src/players.json")
        self.result = self.getResult()

        self.player1.stats.games.append(self.result["player1"])
        self.player2.stats.games.append(self.result["player2"])
        P1 = find(lambda x: x["id"] == self.player1.id, data["players"])
        P2 = find(lambda x: x["id"] == self.player2.id, data["players"])
        index1 = data["players"].index(P1)
        index2 = data["players"].index(P2)

        data["players"][index2] = self.player2.toJSON()
        data["players"][index1] = self.player1.toJSON()
        writeFile(data, "src/players.json")
        return self.result

    def getStatus(self, player):
        if(self.winner is not None):
            if(self.winner.id == player.id):
                return "victory"
            else:
                return "loss"
        else:
            return "draw"

    def getScore(self):
        if(self.getWinner() == "Draw"):
            self.score = {"player1": 0.5, "player2": 0.5}
        elif(self.getWinner() == self.player1.name):
            self.score = {"player1": 1, "player2": 0}
        else:
            self.score = {"player1": 0, "player2": 1}
        return self.score

    def getResult(self):
        return {
            "id": self.number,
            "player1": {
                "game_id": self.number,
                "status": self.getStatus(self.player1),
                "id": self.player1.id,
                "name": self.player1.name,
                "choice_player": self.choice_player1,
                "choice_opponent": self.choice_player2,
                "elo": self.player1.old_elo,
                "expected_score": self.calculate_expected_score(self.player1.elo, self.player2.elo),
                "score": self.getScore()["player1"]
            },
            "player2": {
                "game_id": self.number,
                "status": self.getStatus(self.player2),
                "id": self.player2.id,
                "name": self.player2.name,
                "choice_player": self.choice_player2,
                "choice_opponent": self.choice_player1,
                "elo": self.player2.old_elo,
                "expected_score": self.calculate_expected_score(self.player2.elo, self.player1.elo),
                "score": self.getScore()["player2"]
            },
            "winner": self.getWinner(),
            "loser": self.getLoser(),
            "score": self.getScore()
        }


In [ ]:

class Tournament:
    def __init__(self, name="RSPLP", rounds=10, matches=[]):
        self.name = name
        self.players = self.initializePlayers()
        self.counter = 0
        self.rounds = rounds
        self.matches = matches

    def initializePlayers(self):
        playersData = readFile("src/players.json")
        players = []
        for item in playersData["players"]:
            initial_strategy = item["strategy"]
            next_strategy = item["next_strategy"]
            player = Player(
                STRATEGIES[item["strategy"]](), initial_strategy, next_strategy, item["id"], item["name"], item["elo"], item["constant"])
            players.append(player)
        return players

    def createVersus(self):
        count = 1
        for i in self.players:
            for j in self.players:
                if i != j:
                    if(self.ifExists(i, j)):
                        self.matches.append(Game(count, i, j))
                    count += 1

    def ifExists(self, player1, player2):
        for i in self.matches:
            arr = [i.player1.id, i.player2.id]
            if player1.id in arr and player2.id in arr:
                return False
        return True

    def Start(self):
        for match in self.matches:
            result = match.Play()
            self.addGame(result)

    def getRating(self):
        dataPlayers = readFile("src/players.json")
        rating = []
        for i in dataPlayers["players"]:
            wins = i["stats"]["wins"]
            losses = i["stats"]["losses"]
            draws = i["stats"]["draws"]
            elo = i["elo"]
            name = i["name"]
            rating.append({"name": name, "elo": elo, "wins": wins,
                          "losses": losses, "draws": draws})
        rating.sort(key=lambda item: item['elo'], reverse=True)

        ratingData = readFile("src/rating.json")
        ratingData["rating"] = rating
        writeFile(ratingData, "src/rating.json")

    def addGame(self, game_result):
        data = readFile()
        data["games"].append(game_result)
        writeFile(data, "src/data.json")
        return True



In [ ]:

tornament = Tournament()
tornament.createVersus()
tornament.Start()
tornament.getRating()

In [ ]:

def showRating():
    rating = readFile("src/rating.json")
    elo = []
    wins = []
    losses = []
    name = []
    draws = []
    for i in rating["rating"]:
        elo.append(i["elo"])
        wins.append(i["wins"])
        losses.append(i["losses"])
        name.append(i["name"])
        draws.append(i["draws"])
    df = pd.DataFrame.from_dict(
        {"name": name, "elo": elo, "wins": wins, "losses": losses, "draws": draws})

    print(df)


showRating()